In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [33]:
import pandas as pd
import numpy as np
from pathlib import Path

import pudl
import pudl.extract.eia_bulk_data as ebulk
import pudl.transform.eia_bulk_elec as tbulk

In [4]:
fpath = Path('/home/tpb/catalyst/workspace/data/eia_bulk_data/ELEC.zip')
assert fpath.exists()

In [18]:
%%time
raw = ebulk.extract(fpath)

CPU times: user 1min 26s, sys: 220 ms, total: 1min 26s
Wall time: 1min 26s


In [19]:
meta = raw['metadata']
ts = raw['timeseries']

In [20]:
meta.shape, ts.shape

((8768, 30), (692317, 3))

In [44]:
meta.sample(5)

,series_id,name,units,f,copyright,source,iso3166,lat,lon,geography,start,end,last_updated,latlon,description,geoset_id,category_id,parent_category_id,notes,childseries,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
7744,ELEC.COST_BTU.NG-UT-94.A,Average cost of fossil fuels for electricity g...,dollars per million Btu,A,None,"EIA, U.S. Energy Information Administration",USA-UT,NaN,NaN,USA-UT,2008,2021,2022-02-25T15:25:17-05:00,NaN,Natural Gas; Power plants owned by unregulated...,ELEC.COST_BTU.NG-94.A,NaN,NaN,NaN,NaN,Average cost of fossil fuels for electricity g...,natural gas,Utah,independent power producers (total),annual,COST_BTU,NG,UT,94,A
573,ELEC.COST_BTU.NG-WSC-94.M,Average cost of fossil fuels for electricity g...,dollars per million Btu,M,None,"EIA, U.S. Energy Information Administration",NaN,NaN,NaN,USA-AR+USA-LA+USA-OK+USA-TX,200801,202203,2022-05-24T10:42:22-04:00,NaN,Natural Gas; Power plants owned by unregulated...,ELEC.COST_BTU.NG-94.M,NaN,NaN,NaN,NaN,Average cost of fossil fuels for electricity g...,natural gas,West South Central (total),independent power producers (total),monthly,COST_BTU,NG,WSC,94,M
165,ELEC.COST_BTU.PEL-NV-98.Q,Average cost of fossil fuels for electricity g...,dollars per million Btu,Q,None,"EIA, U.S. Energy Information Administration",USA-NV,NaN,NaN,USA-NV,2008Q1,2022Q1,2022-05-24T10:42:22-04:00,NaN,Summation of all petroleum liquids (distallte ...,ELEC.COST_BTU.PEL-98.Q,NaN,NaN,NaN,NaN,Average cost of fossil fuels for electricity g...,petroleum liquids,Nevada,electric power (total),quarterly,COST_BTU,PEL,NV,98,Q
1102,ELEC.COST_BTU.PEL-IL-94.M,Average cost of fossil fuels for electricity g...,dollars per million Btu,M,None,"EIA, U.S. Energy Information Administration",USA-IL,NaN,NaN,USA-IL,200801,202203,2022-05-24T10:42:22-04:00,NaN,Summation of all petroleum liquids (distallte ...,ELEC.COST_BTU.PEL-94.M,NaN,NaN,NaN,NaN,Average cost of fossil fuels for electricity g...,petroleum liquids,Illinois,independent power producers (total),monthly,COST_BTU,PEL,IL,94,M
920,ELEC.COST_BTU.NG-SAT-97.M,Average cost of fossil fuels for electricity g...,dollars per million Btu,M,None,"EIA, U.S. Energy Information Administration",NaN,NaN,NaN,USA-DC+USA-DE+USA-FL+USA-GA+USA-MD+USA-NC+USA-...,200801,202203,2022-05-24T10:42:22-04:00,NaN,Natural Gas; Power plants in the industrial se...,ELEC.COST_BTU.NG-97.M,NaN,NaN,NaN,NaN,Average cost of fossil fuels for electricity g...,natural gas,South Atlantic (total),all industrial (total),monthly,COST_BTU,NG,SAT,97,M


In [22]:
ts.head(5)

,series_id,date,value
0,ELEC.COST_BTU.COW-PCC-99.M,2022-03-01,NaN
1,ELEC.COST_BTU.COW-PCC-99.M,2022-02-01,NaN
2,ELEC.COST_BTU.COW-PCC-99.M,2022-01-01,NaN
3,ELEC.COST_BTU.COW-PCC-99.M,2021-12-01,NaN
4,ELEC.COST_BTU.COW-PCC-99.M,2021-11-01,NaN


In [23]:
ts.isna().mean()

series_id    0.000000
date         0.000000
value        0.142286
dtype: float64

In [24]:
ts.isna().groupby(ts['series_id'].str[-1])['value'].mean()

series_id
A    0.191406
M    0.139893
Q    0.137875
Name: value, dtype: float64

In [25]:
ts.memory_usage(deep=True) / 2**20

Index        0.000122
series_id    2.278270
date         5.281960
value        5.281960
dtype: float64

In [26]:
meta.describe()

,lat,lon,category_id,parent_category_id
count,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN


In [27]:
meta.loc[:, ~meta.isna().all()].describe()

,series_id,name,units,f,copyright,source,iso3166,geography,start,end,last_updated,description,geoset_id,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
count,8768,8768,8768,8768,8768,8768,6837,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768
unique,8768,8768,2,3,1,1,52,62,37,99,23,594,453,2,7,62,12,3,2,7,62,12,3
top,ELEC.COST_BTU.COW-PCC-99.M,Average cost of fossil fuels for electricity g...,billion Btu,M,None,"EIA, U.S. Energy Information Administration",USA,USA,200801,202203,2022-05-24T10:42:22-04:00,Natural Gas; Power plants owned by companies w...,ELEC.RECEIPTS_BTU.PEL-98.M,Receipts of fossil fuels by electricity plants...,natural gas,United States,electric utility,monthly,RECEIPTS_BTU,NG,US,1,M
freq,1,1,6482,3035,8768,8768,450,450,2949,2798,5453,299,61,6482,2256,450,1637,3035,6482,2256,450,1637,3035


In [29]:
all_nan = meta.isna().all()
all_none = meta.eq('None').all()
to_drop = all_nan | all_none
dropped_col_names = meta.columns[to_drop]

In [31]:
set(dropped_col_names)

{'category_id',
 'childseries',
 'copyright',
 'lat',
 'latlon',
 'lon',
 'notes',
 'parent_category_id'}

In [37]:
tbulk._get_empty_col_names(meta)

{'category_id',
 'childseries',
 'copyright',
 'lat',
 'latlon',
 'lon',
 'notes',
 'parent_category_id'}

In [38]:
tbulk._get_redundant_frequency_col_names(meta)

{'f', 'frequency_code'}

In [41]:
tbulk._get_zero_info_col_names(meta)

{'copyright', 'source'}

In [35]:
meta['fuel_code'].value_counts()

NG     2256
PEL    1811
COW    1741
BIT    1149
SUB     963
PC      637
LIG     211
Name: fuel_code, dtype: int64

In [36]:
meta.columns[meta.nunique() == 1]

Index(['copyright', 'source'], dtype='object')

In [46]:
geo_parts = meta['geoset_id'].str.split('-', expand=True)

In [59]:
tbulk._get_col_names_to_drop(meta)

{'category_id',
 'childseries',
 'copyright',
 'f',
 'frequency_code',
 'geoset_id',
 'lat',
 'latlon',
 'lon',
 'name',
 'notes',
 'parent_category_id',
 'source'}

In [53]:
pd.set_option('display.max_colwidth', None)

In [58]:
meta.sample(5, random_state=2)[['series', 'name', 'description']]

,series,name,description
275,Average cost of fossil fuels for electricity generation (per Btu),Average cost of fossil fuels for electricity generation (per Btu) : petroleum coke : Michigan : electric utility : monthly,Petroleum Coke (solid residual petroluem); Power plants owned by regulated electric utilties;
7717,Average cost of fossil fuels for electricity generation (per Btu),Average cost of fossil fuels for electricity generation (per Btu) : petroleum liquids : Kentucky : electric utility : annual,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by regulated electric utilties;"
8069,Average cost of fossil fuels for electricity generation (per Btu),Average cost of fossil fuels for electricity generation (per Btu) : petroleum liquids : Pacific Noncontiguous (total) : electric utility : annual,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by regulated electric utilties; Summation of the states in the Pacific Noncontiguous census region including Alaska and Hawaii.;"
4344,Receipts of fossil fuels by electricity plants (Btu),Receipts of fossil fuels by electricity plants (Btu) : natural gas : Pacific Contiguous (total) : all sectors : quarterly,"Natural Gas; All sectors; Summation of the states in the Pacific Contiguous census region including California, Oregon, and Washington;"
8561,Receipts of fossil fuels by electricity plants (Btu),Receipts of fossil fuels by electricity plants (Btu) : coal : Connecticut : electric utility cogen : monthly,Summation of all types of coal; Power plants owned by regualted electric utilities that produce both electricity and useful heat;


In [62]:
dmeta = meta.drop(columns=tbulk._get_col_names_to_drop(meta))

In [63]:
dmeta.sample(5, random_state=2)

,series_id,units,iso3166,geography,start,end,last_updated,description,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
275,ELEC.COST_BTU.PC-MI-1.M,dollars per million Btu,USA-MI,USA-MI,200801,202203,2022-05-24T10:42:22-04:00,Petroleum Coke (solid residual petroluem); Power plants owned by regulated electric utilties;,Average cost of fossil fuels for electricity generation (per Btu),petroleum coke,Michigan,electric utility,monthly,COST_BTU,PC,MI,1,M
7717,ELEC.COST_BTU.PEL-KY-1.A,dollars per million Btu,USA-KY,USA-KY,2008,2021,2022-02-25T15:25:17-05:00,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by regulated electric utilties;",Average cost of fossil fuels for electricity generation (per Btu),petroleum liquids,Kentucky,electric utility,annual,COST_BTU,PEL,KY,1,A
8069,ELEC.COST_BTU.PEL-PCN-1.A,dollars per million Btu,NaN,USA-AK+USA-HI,2008,2021,2022-02-25T15:25:17-05:00,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by regulated electric utilties; Summation of the states in the Pacific Noncontiguous census region including Alaska and Hawaii.;",Average cost of fossil fuels for electricity generation (per Btu),petroleum liquids,Pacific Noncontiguous (total),electric utility,annual,COST_BTU,PEL,PCN,1,A
4344,ELEC.RECEIPTS_BTU.NG-PCC-99.Q,billion Btu,NaN,USA-CA+USA-OR+USA-WA,2008Q1,2022Q1,2022-05-24T10:42:22-04:00,"Natural Gas; All sectors; Summation of the states in the Pacific Contiguous census region including California, Oregon, and Washington;",Receipts of fossil fuels by electricity plants (Btu),natural gas,Pacific Contiguous (total),all sectors,quarterly,RECEIPTS_BTU,NG,PCC,99,Q
8561,ELEC.RECEIPTS_BTU.COW-CT-3.M,billion Btu,USA-CT,USA-CT,200801,201212,2016-12-19T17:19:30-05:00,Summation of all types of coal; Power plants owned by regualted electric utilities that produce both electricity and useful heat;,Receipts of fossil fuels by electricity plants (Btu),coal,Connecticut,electric utility cogen,monthly,RECEIPTS_BTU,COW,CT,3,M


In [68]:
wide_ts = ts.merge(dmeta[['series_id', 'series_code', 'fuel', 'region','sector', 'frequency']], on='series_id').drop(columns='series_id')

In [75]:
wide_ts = wide_ts.set_index(['series_code', 'fuel', 'region','sector', 'frequency', 'date']).unstack('series_code')

In [80]:
wide_ts.columns = wide_ts.columns.droplevel(level=None)

In [81]:
wide_ts.sample(5)

,,,,series_code,COST_BTU,RECEIPTS_BTU
fuel,region,sector,frequency,date,,
bituminous coal,Pacific Noncontiguous (total),electric utility cogen,monthly,2018-10-01,NaN,0.00000
petroleum liquids,East South Central (total),independent power producers (total),monthly,2009-12-01,NaN,NaN
petroleum coke,California,electric power (total),monthly,2013-08-01,0.0000,0.00000
coal,New Mexico,electric utility,monthly,2010-03-01,2.1129,17021.86833
petroleum coke,West South Central (total),industrial cogen,quarterly,2016-01-01,NaN,0.00000
